In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
#from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.linear_model import LinearRegression

## Data loading
------------------------

In [2]:
file="Churn_Modelling.csv"
data=pd.read_csv(file)

In [3]:
#print(data.head(10))
type(data)

pandas.core.frame.DataFrame

## Data cleaning
------------------------

In [4]:
NewData=data.iloc[:,3:]
#NewData=NewData.replace({'Exited':{0:'NotExited', 1:'Exited'}})
#NewData=NewData.replace({'Gender':{'Male':1, 'Female':0}})
NewData.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
NewData.info()
NewData.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [6]:
print(NewData.shape)
print(type(NewData))

(10000, 11)
<class 'pandas.core.frame.DataFrame'>


In [7]:
X=NewData.iloc[:,:-1]
Y=NewData.iloc[:,-1]
print(type(X))
print(X.shape)
print(Y.shape)

<class 'pandas.core.frame.DataFrame'>
(10000, 10)
(10000,)


In [8]:
#label encoder
#obj_df = NewData.select_dtypes(include=['object']).copy()
#obj_df.head()
encoder = LabelEncoder()
X.iloc[:,1] = encoder.fit_transform(X.iloc[:,1])
X.iloc[:,2] = encoder.fit_transform(X.iloc[:,2])
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10


In [9]:
#OneHot Encoder
#hotencoder=OneHotEncoder(categorical_features = [1])
from keras.utils import to_categorical
encodedGEO = pd.DataFrame(to_categorical(X.iloc[:, 1]))
#X = hotencoder.fit_transform(X).toarray()
print(encodedGEO.head())
print(X.head())
X = pd.concat([encodedGEO, X], axis = 1)
X.head()
#X.shape

C:\Users\720S\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


     0    1    2
0  1.0  0.0  0.0
1  0.0  0.0  1.0
2  1.0  0.0  0.0
3  1.0  0.0  0.0
4  0.0  0.0  1.0
   CreditScore  Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0          619          0       0   42       2       0.00              1   
1          608          2       0   41       1   83807.86              1   
2          502          0       0   42       8  159660.80              3   
3          699          0       0   39       1       0.00              2   
4          850          2       0   43       2  125510.82              1   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               1        101348.88  
1          0               1        112542.58  
2          1               0        113931.57  
3          0               0         93826.63  
4          1               1         79084.10  


,0,1,2,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,1.0,0.0,0.0,619,0,0,42,2,0.00,1,1,1,101348.88
1,0.0,0.0,1.0,608,2,0,41,1,83807.86,1,0,1,112542.58
2,1.0,0.0,0.0,502,0,0,42,8,159660.80,3,1,0,113931.57
3,1.0,0.0,0.0,699,0,0,39,1,0.00,2,0,0,93826.63
4,0.0,0.0,1.0,850,2,0,43,2,125510.82,1,1,1,79084.10


In [10]:
#print(X[0])
#NewData.head(1)
#here we know that the first 3 columns was geography, we can remove the first column
#while if the 2nd and 3rd colums is 0 means 1st colum is 1
# we can reduce the dimension
X=X.drop(['Geography',0],1)
X.head()

,1,2,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,0.0,0.0,619,0,42,2,0.00,1,1,1,101348.88
1,0.0,1.0,608,0,41,1,83807.86,1,0,1,112542.58
2,0.0,0.0,502,0,42,8,159660.80,3,1,0,113931.57
3,0.0,0.0,699,0,39,1,0.00,2,0,0,93826.63
4,0.0,1.0,850,0,43,2,125510.82,1,1,1,79084.10


In [11]:
X.shape

(10000, 11)

## LM model does not work* let's try ANN 
Adjust the spliting data process using sklearn

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train,Y_test = train_test_split(X,Y, test_size = 0.2,random_state=0)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(8000, 11)
(8000,)
(2000, 11)
(2000,)


In [13]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

C:\Users\720S\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype float32, int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\720S\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype float32, int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\720S\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype float32, int32, int64, float64 were all converted to float64 by StandardScaler.
  """


In [14]:
import keras 

In [15]:
from keras.models import Sequential
from keras.layers import Dense

In [16]:
classifier = Sequential()

In [17]:
# add input layer
#using random_normal: the accaracy is lower than 0.3
classifier.add(Dense(input_dim=11,units=6,kernel_initializer='uniform', activation='relu'))

In [18]:
# add a second input layer
classifier.add(Dense(units=6,kernel_initializer='uniform', activation='relu'))

In [19]:
#add outer layer
classifier.add(Dense(output_dim=1,kernel_initializer='uniform', activation='softmax'))

C:\Users\720S\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", activation="softmax", units=1)`
  


In [20]:
classifier.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['accuracy']) 

In [21]:
#train the data
classifier.fit(x=X_train,y=Y_train, epochs=20, batch_size=10)

Epoch 1/20
8000/8000 [==============================] - 1s 153us/step - loss: 12.6901 - acc: 0.2040
Epoch 2/20
8000/8000 [==============================] - 1s 108us/step - loss: 12.6901 - acc: 0.2040
Epoch 3/20
8000/8000 [==============================] - 1s 110us/step - loss: 12.6901 - acc: 0.2040
Epoch 4/20
8000/8000 [==============================] - 1s 111us/step - loss: 12.6901 - acc: 0.2040
Epoch 5/20
8000/8000 [==============================] - 1s 116us/step - loss: 12.6901 - acc: 0.2040
Epoch 6/20
8000/8000 [==============================] - 1s 113us/step - loss: 12.6901 - acc: 0.2040
Epoch 7/20
8000/8000 [==============================] - 1s 138us/step - loss: 12.6901 - acc: 0.2040
Epoch 8/20
8000/8000 [==============================] - 1s 140us/step - loss: 12.6901 - acc: 0.2040
Epoch 9/20
8000/8000 [==============================] - 1s 118us/step - loss: 12.6901 - acc: 0.2040
Epoch 10/20
8000/8000 [==============================] - 1s 125us/step - loss: 12.6901 - acc: 0.2040

In [22]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [23]:
y_pred

array([[ True],
       [ True],
       [ True],
       ...,
       [ True],
       [ True],
       [ True]])

## try kmean clustering

In [24]:
from sklearn.cluster import KMeans
k_means = KMeans(n_clusters=2)
k_means.fit(X_train)
#print(k_means.labels_[:])
#print(Y_train[:])

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [25]:
from sklearn.metrics import accuracy_score
score = accuracy_score(Y_train,k_means.predict(X_train))
print('Accuracy:{0:f}'.format(score))
score = accuracy_score(Y_test,k_means.predict(X_test))
print('Accuracy:{0:f}'.format(score))

Accuracy:0.288375
Accuracy:0.305500


accuracy too low and unstable

## try logistic regression 

In [26]:
import sklearn as sk
from sklearn.linear_model import LogisticRegression
import pandas as pd
import os

In [27]:
LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X_train, Y_train)

In [28]:
score = accuracy_score(Y_train,LR.predict(X_train))
print('Accuracy:{0:f}'.format(score))

Accuracy:0.808500


In [29]:
score = accuracy_score(Y_test,LR.predict(X_test))
print('Accuracy:{0:f}'.format(score))

Accuracy:0.811000


## try Support Vector Machines

In [30]:
from sklearn import svm
SVM = svm.LinearSVC()
SVM.fit(X_train, Y_train)
score = accuracy_score(Y_test,SVM.predict(X_test))
print('Accuracy:{0:f}'.format(score))

Accuracy:0.812000


## try Random forest classifier

In [31]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
RF.fit(X_train, Y_train)
score = accuracy_score(Y_test,RF.predict(X_test))
print('Accuracy:{0:f}'.format(score))

Accuracy:0.809500


## try Neural Networkd
try 3 different solver. with activation default 'relu': linear unit function. 

In [62]:
from sklearn.neural_network import MLPClassifier
#lbfgs: optimizer in the family of quasi-Newton methods
NN = MLPClassifier(solver='lbfgs', hidden_layer_sizes=6, max_iter=100000)
NN.fit(X_train, Y_train)
score = accuracy_score(Y_test,NN.predict(X_test))
print('lbfgs Accuracy:{0:f}'.format(score))

#stochastic gradient descent.
NN = MLPClassifier(solver='sgd', hidden_layer_sizes=6)
NN.fit(X_train, Y_train)
score = accuracy_score(Y_test,NN.predict(X_test))
print('sgd Accuracy:{0:f}'.format(score))

#stochastic gradient-based optimizer proposed by Kingma, Diederik, and Jimmy Ba
NN = MLPClassifier(solver='adam', hidden_layer_sizes=6)
NN.fit(X_train, Y_train)
score = accuracy_score(Y_test,NN.predict(X_test))
print('adam Accuracy:{0:f}'.format(score))

lbfgs Accuracy:0.859500


C:\Users\720S\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


sgd Accuracy:0.852500
adam Accuracy:0.862000


## Spliting data
------------------------
use k-fold technique to resample the data set

In [33]:
#import seaborn as sns
#sns.pairplot(NewData)
kf= KFold(n_splits=3)
kf.get_n_splits(X)
print(kf)

KFold(n_splits=3, random_state=None, shuffle=False)


In [34]:
#for train_index, test_index in kf.split(X):
    #X_train, X_test = X[train_index], X[test_index]
    #Y_train, Y_test = Y[train_index], Y[test_index]

In [35]:
#print(X_train.shape)
#print(Y_train.shape)
#print(X_test.shape)
#print(Y_test.shape)

## build the model using Linear regression
---------------------------

In [36]:
lm = LinearRegression().fit(X_train, Y_train)
print('slope: ',lm.coef_)
print('intercept: ',lm.intercept_)
predictions = lm.predict(X_test)
predictions[0:5]

slope:  [ 0.05804786  0.00618424 -0.00992217 -0.03833442  0.11544162 -0.01060152
  0.01767596 -0.01299554 -0.00268823 -0.06836955  0.00426653]
intercept:  0.204


array([0.26241452, 0.33639512, 0.18921297, 0.14217021, 0.20073383])

In [38]:
#Y_test[2]
#abs(predictions[0])
#if(abs(Y_test[0]-predictions[0]))<0.5:
#    print('yes')

In [39]:
#nbcorrect=0
#for i in Y_test,predictions:
#    if(abs(Y_test[0]-predictions[0]))<0.5:
#        nbcorrect=nbcorrect+1
#print(nbcorrect)

## conclusion

Although most of the people use ANN to perform this exercise and I've also tries to use it tunning the parameters, but I don't know why I didn't get the good result out of it.

I've also tried different method like K means,linear model, etc... eventually, I decided to use the simple classification method to perform the result. However, it seems that logistic regression, SVM, Random Forest, can have up to 81% of the accuracy. Neural Network got the best accuracy up to 86%.

Therefore my conclusion of the classifier would use the Neural Network method. But the parameter still needs to be tune since there's the convergence warning.